In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_path = '../nhat_hoang/Dataset/Dataset2/data/'

In [3]:
items = pd.read_csv(data_path+'items.csv')
items.head()

,Unnamed: 0,item_feature_2,item_feature_3,item_feature_4,item_feature_5,item_feature_6,item_feature_7,item_feature_8,item_feature_9,item_feature_10,item_feature_11,item_id,item_feature_1
0,4,0,0,1,0,0,1,0,0,1,0,14293,0
1,7,0,0,1,0,0,0,0,1,1,0,11959,1
2,8,0,0,1,0,0,0,0,1,1,0,3590,2
3,15,1,0,1,0,0,0,0,0,0,22500,10093,1
4,16,1,0,0,0,0,0,0,0,0,12500,7561,0


In [4]:
transactions = pd.read_json(data_path+'transactions.json')
transactions.head()

,user_feature_1,user_feature_2,user_feature_3,user_feature_4,serverdate,user_id,item_id,style_0,style_1,style_2,...,style_51,style_52,style_53,style_54,style_55,style_56,style_57,style_58,style_59,style_60
0,-1,-1,-1,-1,1619534246606,280718,9067,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-1,-1,-1,-1,1619534263230,339313,5877,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1,-1,-1,-1,1619526257261,640560,14614,0,0,160,...,0,0,0,0,0,0,0,0,0,0
3,-1,-1,-1,-1,1619534249835,473846,14364,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-1,-1,-1,-1,1619534257609,586986,13644,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Transform the dataset for Slirec model

In [ ]:
transformed_transactions = transactions.copy()
# drop unused columns, sort by timestamp
transformed_transactions.drop(transactions.columns.difference(['user_id', 'item_id', 'serverdate']), 1, inplace=True)
transformed_transactions.sort_values(by='serverdate', ignore_index=True, inplace=True)
# merge the item_feature_1 
transformed_transactions = transformed_transactions.merge(items[['item_id', 'item_feature_1']].drop_duplicates(), left_on="item_id", right_on="item_id")
transformed_transactions.rename(columns={'item_feature_1': 'cate_id', 'serverdate': 'timestamp'}, inplace=True)
# rearrange the columns
transformed_transactions = transformed_transactions.reindex(columns=['user_id', 'item_id', 'timestamp', 'cate_id'])

In [14]:
transformed_transactions.to_csv(os.path.join("..", "nhat_hoang", "Dataset", "Dataset2", "data", "inputcsv"), sep="\t", index=None, header=None)

### Original dataset split

In [12]:
dfp_transactions = transformed_transactions.copy()

# Determine the percentage of last user transaction to select to get 10% test data
N = dfp_transactions['user_id'].count()
print("Total of {} transactions".format(N))

# Train/Test split percentage
test_prct=0.1
train_prct=1-test_prct

M = int(test_prct*N)
print("Need to select {} rows for the test set".format(M))

Total of 5205410 transactions
Need to select 520541 rows for the test set


In [14]:
# Split data
dfp_transactions_sorted = dfp_transactions.sort_values('timestamp')
dfp_test_data = dfp_transactions_sorted.tail(M)
dfp_train_data = dfp_transactions_sorted.drop(dfp_test_data.index)

print("test contains {} samples".format(dfp_test_data['user_id'].count()))
print("train contains {} samples".format(dfp_train_data['user_id'].count()))

test contains 520541 samples
train contains 4684869 samples


In [21]:
# Save this data for later use
transformed_data_dir = os.path.join("..", "nhat_hoang", "Dataset", "Dataset2", "transformed_data")

dfp_train_data.to_csv(os.path.join(transformed_data_dir, 'train_data'), sep="\t", index=None, header=None)
dfp_test_data.to_csv(os.path.join(transformed_data_dir, 'test_data'), sep="\t", index=None, header=None)